In [1]:
from activ import load_data, data_normalization
import os
from activ.readfile import TrackTBIFile
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
import numpy as np
from activ.analytics import heatmap, nmf_bases_heatmap
from activ.clustering import UmapClusteringResults
from os.path import dirname, join
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import display, Latex, Markdown
from pylatex import Document, Section, Subsection, Command
from pylatex import Document, Section, Figure, SubFigure, NoEscape
from pylatex.utils import italic, NoEscape
from scipy.stats import norm, sem

/Users/ahyeon/anaconda3/envs/activ/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [15]:
nmfdata = load_data(uoinmf=True)

raw = load_data()
rawbm = raw.biomarkers #(247,235)
rawoc = raw.outcomes
oc_feature_names = raw.outcome_features
bm_feature_names = raw.biomarker_features



In [16]:
def drop_x_percent(A,w,h,feature_names,percent):
    total_features = h.shape[1]
    num_features_to_drop = np.round(total_features*percent//100)
    h_sum = np.sum(h, axis=0)
    h_sum_sorted = np.argsort(h_sum)
    indices_lowest_values = h_sum_sorted[:num_features_to_drop]
    dropped_features = sorted(feature_names[indices_lowest_values])
    h_new = np.delete(h, indices_lowest_values, axis=1)
    A_new = np.delete(A, indices_lowest_values, axis=1) 
    return A_new, h_new, dropped_features

In [17]:
# drop outcomes
A_oc = data_normalization.data_normalization(rawoc, 'positive') #(247,177)
w_oc = nmfdata.outcomes #(247,6)
h_oc = nmfdata.outcome_bases #(6,177)

percent = [0,10,20,30,40,50,60,70,80,90]
for p in percent:
    A_oc_features_dropped, h_new, dropped_features = drop_x_percent(A_oc, w_oc, h_oc, oc_feature_names, p)
    fname = '/Users/ahyeon/data/activ/no_transform/oc_features_dropped_{}.h5'.format(p)
    TrackTBIFile.write(fname, rawbm, A_oc_features_dropped)

In [18]:
# drop biomarkers
A_bm = data_normalization.data_normalization(rawbm, 'positive') #(247,177)
w_bm = nmfdata.biomarkers #(247,6)
h_bm = nmfdata.biomarker_bases #(6,177)

percent = [0,10,20,30,40,50,60,70,80,90]
for p in percent:
    A_bm_features_dropped, h_new, dropped_features = drop_x_percent(A_bm, w_bm, h_bm, bm_feature_names, p)
    fname = '/Users/ahyeon/data/activ/no_transform/bm_features_dropped_{}.h5'.format(p)
    TrackTBIFile.write(fname, A_bm_features_dropped, rawoc)